In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
pip install keybert 

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: keybert in c:\users\97252\anaconda3\lib\site-packages (0.8.5)



In [2]:
pip install whoosh transformers faiss-gpu torch pinecone sentence_transformers

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [3]:
pip install whoosh

Note: you may need to restart the kernel to use updated packages.


In [6]:
# import json
# from sentence_transformers import SentenceTransformer
# import numpy as np
# from transformers import AutoTokenizer, AutoModel
# import torch
# # Load your embedding model
# model = SentenceTransformer("all-MiniLM-L6-v2")
# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")


# def load_and_embed_dataset(dataset_path, prompt_field="prompt", completion_field="completion"):
#     """
#     Load dataset from JSONL, truncate metadata to fit byte limits, and embed texts.
#     Args:
#         dataset_path (str): Path to the JSONL file with "prompt" and "completion".
#         prompt_field (str): Field name for prompt.
#         completion_field (str): Field name for completion.
#     Returns:
#         embeddings (np.ndarray): Embedded vectors.
#         metadata (list): List of dictionaries with truncated "prompt" and "completion".
#     """
#     print("Loading dataset and embedding texts...")

#     embeddings = []
#     metadata = []

#     with open(dataset_path, 'r') as f:
#         for line in f:
#             data = json.loads(line)

#             # Get and truncate prompt and completion to fit within metadata size limits
#             prompt = truncate_text_to_bytes(data.get(prompt_field, ""))
#             completion = truncate_text_to_bytes(data.get(completion_field, ""))

#             # Concatenate prompt and completion for embedding
#             text_to_embed = f"{prompt} {completion}"
#             embedding = model.encode(text_to_embed)

#             # Append embedding and metadata to lists
#             embeddings.append(embedding)
#             metadata.append({"prompt": prompt, "completion": completion})

#     # Convert embeddings list to numpy array
#     embeddings = np.array(embeddings)
#     print("Embedding and truncation complete.")

#     return embeddings, metadata

# def embed_text(text):
#     # Tokenize and embed the text
#     model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

#     inputs = tokenizer(text, return_tensors="pt", truncation=True)
#     with torch.no_grad():
#         embeddings = model(**inputs).pooler_output  # use the pooled output as the embedding
#     return embeddings[0].cpu().numpy()

In [5]:
import os
import json
import numpy as np
from pinecone import Pinecone, ServerlessSpec
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from whoosh.index import create_in, open_dir, exists_in
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser, MultifieldParser,OrGroup
from whoosh import analysis, scoring
from keybert import KeyBERT
import uuid

def truncate_text_to_bytes(text, max_bytes=40960):
    """Truncate text to fit within a specified byte limit."""
    # Start with an initial truncation within a reasonable character limit
    truncated_text = text[:15000]
    # Iteratively reduce size if byte count exceeds the limit
    while len(truncated_text.encode('utf-8')) > max_bytes:
        truncated_text = truncated_text[:-100]  # Remove more characters
    return truncated_text


class RetrievalService:
    def __init__(self, index_dir, pinecone_index_name, pinecone_api_key, pinecone_environment):
        """
        Initialize the RetrievalService.

        :param index_dir: Directory path to store Whoosh index for sparse retrieval.
        :param pinecone_index_name: Name of the Pinecone index for dense retrieval.
        :param pinecone_api_key: Your Pinecone API key.
        :param pinecone_environment: Pinecone environment region.
        """
        self.index_dir = index_dir
        self.pinecone_index_name = pinecone_index_name

        # Initialize sentence-transformers model for dense embeddings
        self.model = SentenceTransformer('intfloat/e5-large')
        self.embedding_dim = 1024
        # Initialize Pinecone
        self.pc=Pinecone(api_key=pinecone_api_key)

        # Create Pinecone index if it doesn't exist
        print("Creating a Pinecone index...")
        existing_indexes = [index_info["name"] for index_info in self.pc.list_indexes()]
        if self.pinecone_index_name not in existing_indexes:
             self.pc.create_index(
                name=self.pinecone_index_name,
                dimension=self.embedding_dim,
                metric="cosine",
                spec=ServerlessSpec(
                    cloud="aws",
                    region="us-east-1"
                )
            )
        self.pinecone_index=self.pc.Index(self.pinecone_index_name)

        # Initialize or create Whoosh index
        if not os.path.exists(index_dir):
            os.makedirs(index_dir)
            schema = Schema(doc_id=ID(stored=True, unique=True), content=TEXT(stored=True, analyzer=analysis.StemmingAnalyzer()))
            self.index_sparse = create_in(index_dir, schema)
        elif not exists_in(index_dir):
            schema = Schema(doc_id=ID(stored=True, unique=True), content=TEXT(stored=True, analyzer=analysis.StemmingAnalyzer()))
            self.index_sparse = create_in(index_dir, schema)
        else:
            self.index_sparse = open_dir(index_dir)



    def index_documents(self, jsonl_file, sparse=True, dense=True, batch_size=64):
        # Read documents
        documents = []
        doc_ids = []
        metadata = []
        texts_to_embed = []

        with open(jsonl_file, 'r') as f:
            for line in f:
                data = json.loads(line)
                prompt = truncate_text_to_bytes(data.get("prompt", ""))#TODO Check if can be deleted ( the trucatnion)
                completion = truncate_text_to_bytes(data.get("completion", ""))
                document = f"Prompt: {prompt}\nCompletion: {completion}"
                # Generate a unique UUID for doc_id
                doc_id = str(uuid.uuid4())
                documents.append((doc_id, document))
                doc_ids.append(doc_id)
                metadata.append({"prompt": prompt, "completion": completion})
                texts_to_embed.append(f"{prompt} {completion}")

        if sparse:
            print("Indexing documents for sparse retrieval using Whoosh...")
            self.index_sparse_documents(documents)

        if dense:
            print("Indexing documents for dense retrieval using Pinecone...")
            self.index_dense_documents(doc_ids, texts_to_embed, metadata, batch_size=batch_size)



    def index_sparse_documents(self, documents):
        """
        Index documents into a sparse (BM25) Whoosh index.

        :param documents: List of (doc_id, content) tuples.
        """
        writer = self.index_sparse.writer()
        for doc_id, content in documents:
            writer.add_document(doc_id=doc_id, content=content)
            print(f"Indexed Document: DocID={doc_id}, Content (excerpt)={content[:100]}...")  # Debug statement
        writer.commit()
        print("Sparse indexing completed.")
    def index_dense_documents(self, doc_ids, texts_to_embed, metadata, batch_size=64):
        """
        Index documents into Pinecone using provided doc_ids, texts, and metadata.

        :param doc_ids: List of document IDs.
        :param texts_to_embed: List of texts to embed.
        :param metadata: List of metadata dictionaries.
        :param batch_size: Batch size for processing documents.
        """
        # Encode texts using sentence-transformers
        embeddings = self.model.encode(texts_to_embed, convert_to_numpy=True, batch_size=batch_size)

        # Prepare list of (id, vector, metadata) dictionaries for upsert
        to_upsert = []
        for doc_id, embedding, meta in zip(doc_ids, embeddings, metadata):
            to_upsert.append({
                "id": doc_id,
                "values": embedding.tolist(),
                "metadata": meta
            })

        # Upsert in batches using tqdm for progress tracking
        print("Upserting the embeddings to the Pinecone index...")
        for i in tqdm(range(0, len(to_upsert), batch_size)):
            i_end = min(i + batch_size, len(to_upsert))
            batch = to_upsert[i:i_end]
            self.pinecone_index.upsert(vectors=batch)
        print("Dense indexing completed.")

        

    def extract_keyphrases(self,kw_model,query, num_keywords=5):
        """Extract significant words or keyphrases from the query."""
        keywords = kw_model.extract_keywords(query, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=num_keywords)
        # Extract only the words from the keywords
        return [word for word, score in keywords]

    def search_sparse(self, query, top_k=10):
        try:
            kw_model = KeyBERT()
            keyphrases = self.extract_keyphrases(kw_model,query)
            keyphrase_query = " OR ".join(keyphrases)
                    
            with self.index_sparse.searcher(weighting=scoring.BM25F()) as searcher:
                parser = QueryParser("content", self.index_sparse.schema, group=OrGroup)
                parsed_query = parser.parse(keyphrase_query)
                print(f"Parsed Query: {parsed_query}")
                results = searcher.search(parsed_query, limit=top_k)
                print(f"Number of results found: {len(results)}")
                hits = [(hit['doc_id'], hit.score) for hit in results]
            return hits
        except Exception as e:
            print(f"An error occurred during sparse search: {e}")
            return []







    def search_dense(self, query, top_k=10):
        """
        Perform a dense search using Pinecone.

        :param query: The search query as a string.
        :param top_k: Number of top results to retrieve.
        :return: List of (doc_id, score, metadata) tuples.
        """
        # Generate the embedding for the query question
        query_embedding = self.model.encode(query, convert_to_numpy=True)

        # query_embedding = embed_text(query)

        # Perform the query with Pinecone
        try:
            response = self.pinecone_index.query(
                vector=query_embedding.tolist(),
                top_k=top_k,
                include_metadata=True
            )
            # Extract the matches with metadata
            matches = response['matches']
            hits = [(match['id'], match['score']) for match in matches]
        except Exception as e:
            print(f"Error querying index: {e}")
            hits = []

        return hits

    def search(self, query, top_k=10, alpha=0.5):
        """
        Perform a hybrid search combining sparse and dense retrieval.

        :param query: The search query as a string.
        :param top_k: Number of top results to retrieve.
        :param alpha: Weighting factor for dense scores (between 0 and 1).
        :return: List of (doc_id, combined_score) tuples.
        """
        # Perform sparse and dense searches
        sparse_hits = self.search_sparse(query, top_k=top_k)
        dense_hits = self.search_dense(query, top_k=top_k)

        # Normalize scores
        max_sparse_score = max([score for _, score in sparse_hits], default=1)
        max_dense_score = max([score for _, score in dense_hits], default=1)
        sparse_scores = {doc_id: score / max_sparse_score for doc_id, score in sparse_hits}
        dense_scores = {doc_id: score / max_dense_score for doc_id, score in dense_hits}

        # Combine results
        all_doc_ids = set(sparse_scores.keys()) | set(dense_scores.keys())
        combined_scores = {}
        for doc_id in all_doc_ids:
            sparse_score = sparse_scores.get(doc_id, 0)
            dense_score = dense_scores.get(doc_id, 0)
            combined_score = (1 - alpha) * sparse_score + alpha * dense_score
            combined_scores[doc_id] = combined_score

        # Sort and return top_k results
        sorted_results = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
        return sorted_results[:top_k]

    def retrieve_document_by_id(self, doc_id):
        """
        Retrieve the actual document text given a document ID.

        :param doc_id: The ID of the document to retrieve.
        :return: The document content.
        """
        # First, try to retrieve from Whoosh index
        with self.index_sparse.searcher() as searcher:
            doc = searcher.document(doc_id=doc_id)
            if doc:
                return doc['content']
        
        # If not found in Whoosh, try to retrieve from Pinecone metadata
        try:
            response = self.pinecone_index.fetch(ids=[doc_id])
            vectors = response.get('vectors', {})
            if doc_id in vectors:
                metadata = vectors[doc_id].get('metadata', {})
                # Reconstruct the document content from metadata
                prompt = metadata.get('prompt', '')
                completion = metadata.get('completion', '')
                content = f"Prompt: {prompt}\nCompletion: {completion}"
                return content
        except Exception as e:
            print(f"Error retrieving document by ID from Pinecone: {e}")
        
        return None



In [7]:
# Replace these with your Pinecone details
PINECONE_API_KEY = ''
PINECONE_ENVIRONMENT = 'us-east-1'  # e.g., 'us-west1-gcp'
PINECONE_INDEX_NAME = 'trumpindex'

# Initialize the retrieval service
retrieval_service = RetrievalService(
    index_dir='whoosh_index',
    pinecone_index_name=PINECONE_INDEX_NAME,
    pinecone_api_key=PINECONE_API_KEY,
    pinecone_environment=PINECONE_ENVIRONMENT,
    
    # encoder_name='sentence-transformers/all-MiniLM-L6-v2'
)

# # Create a Pinecone index
# create_pinecone_index(PINECONE_INDEX_NAME, dimension=384)




Creating a Pinecone index...


In [146]:

# Index documents from a JSONL file (if not already done)

retrieval_service.index_documents('Trump_DB.jsonl', sparse=True, dense=False)

# Perform a hybrid search


Indexing documents for sparse retrieval using Whoosh...
Indexed Document: DocID=b8e92983-1f23-4220-bf48-09b91943dfc8, Content (excerpt)=Prompt: Mr. President, welcome back and welcome to election day.
Completion: Well, thank you very mu...
Indexed Document: DocID=98592dc4-c454-42d9-93e3-dbdd8af87b1b, Content (excerpt)=Prompt: Yeah. Yeah. He's going through some very important treatment [crosstalk] and I know he's goi...
Indexed Document: DocID=cd29e97e-0392-49e6-ad55-5ae8caf63ccc, Content (excerpt)=Prompt: Absolutely. When you spar for a living in an octagon to relax, you are a tough guy. Meanwhil...
Indexed Document: DocID=6c5f8bc8-17aa-4481-b1f9-efb04bc3cb88, Content (excerpt)=Prompt: It's huge.
Completion: The crowds have been incredible, and nobody's ever had it before. And...
Indexed Document: DocID=5cc8a068-0832-44df-ad94-9287ae1c23a2, Content (excerpt)=Prompt: Right.
Completion: But the second debate in particular, I think was something that worked ou...
Indexed Document: DocID=

In [9]:
query = "What are your views on the economy?"
results = retrieval_service.search(query, top_k=10, alpha=0.5)

# Retrieve and print documents
for doc_id, score in results:
    content = retrieval_service.retrieve_document_by_id(doc_id)
    print(f"DocID: {doc_id}, Score: {score}")
    print(f"Content:\n{content}\n")

Parsed Query: (content:economi OR content:view)
Number of results found: 182
DocID: e9f78594-9aac-4f4e-bc5d-d1638616c8ac, Score: 0.5
Content:
Prompt: ——if you view it the same way. And do you view the U.S. alliance system shifting under your Presidency, away from——
Completion: Who are you with, out of curiosity?

DocID: 99a41424-692f-4524-a306-8ee009930839, Score: 0.5
Content:
Prompt: Can I ask you about that?
Completion: Our, our economy is good.

DocID: b85b9035-1160-4ec3-a72b-ea4a352603dd, Score: 0.48965987590339877
Content:
Prompt: We're going to talk a lot about the economy.
Completion: Sure.

DocID: 1a1d1a74-f05f-4d08-af2c-b9472d07e661, Score: 0.48626725942213633
Content:
Prompt: -- the economy's great.
Completion: I had a choice. I could have let it implode and killed it or I could have managed it --

DocID: cf556af9-0e6e-4ead-9cf7-111dfe18dc28, Score: 0.4858222398605607
Content:
Prompt: Where will the economy be by the end of this year in your view? You have 20 seconds.
Complet

In [149]:
# Perform a dense search
query = "What are your views on the economy?"
dense_results = retrieval_service.search_dense(query, top_k=10)

# Retrieve and print documents
for doc_id, score in dense_results:
    content = retrieval_service.retrieve_document_by_id(doc_id)
    print(f"DocID: {doc_id}, Score: {score}")
    print(f"Content:\n{content}\n")


DocID: 99a41424-692f-4524-a306-8ee009930839, Score: 0.637638092
Content:
Prompt: Can I ask you about that?
Completion: Our, our economy is good.

DocID: b85b9035-1160-4ec3-a72b-ea4a352603dd, Score: 0.624451578
Content:
Prompt: We're going to talk a lot about the economy.
Completion: Sure.

DocID: 1a1d1a74-f05f-4d08-af2c-b9472d07e661, Score: 0.620125055
Content:
Prompt: -- the economy's great.
Completion: I had a choice. I could have let it implode and killed it or I could have managed it --

DocID: 8c00ba5d-a1df-40ab-bc44-414cb4615023, Score: 0.556632459
Content:
Prompt: The, currently things are, the economy's pretty hot. Running pretty hot. Hopefully not too hot—
Completion: No, but I think it's maybe as good as it's ever been ever. I mean companies… What I really am happy about is, a lot of cash is coming back in through the billions and trillions of dollars as you know that was overseas where companies were locked and they couldn't get it back. Now it's coming back hundreds of mill

In [147]:
# Perform a sparse search
query = "What are your views on the economy?"

sparse_results = retrieval_service.search_sparse(query, top_k=10)
print(sparse_results)
# Retrieve and print documents
for doc_id, score in sparse_results:
    content = retrieval_service.retrieve_document_by_id(doc_id)
    print(f"DocID: {doc_id}, Score: {score}")
    print(f"Content:\n{content}\n")


Parsed Query: (content:economi OR content:view)
Number of results found: 182
[('e9f78594-9aac-4f4e-bc5d-d1638616c8ac', 9.390654990355637), ('cf556af9-0e6e-4ead-9cf7-111dfe18dc28', 9.124378082344656), ('07eabc4f-dd59-4c14-bd0b-188dd0549585', 9.064165418472472), ('84068df6-cd66-404a-94a1-e3fb9aba5d4e', 8.710835798500547), ('154669b1-9580-46cc-ab43-3d1eaecc7788', 7.842533315960166), ('eae5a2ef-f6e3-40ae-a214-804e6de6b85d', 7.8107309820076125), ('ffb30f84-ff32-483e-94da-4351ce69b398', 7.291864477659173), ('1f7f6874-0f82-402e-9663-1c4bf6cb609c', 7.139922291100996), ('16fd63a6-ab9b-491f-ae3b-510fe443ebc1', 7.139922291100996), ('41f380a4-c448-45ad-ab0c-d53a9270274b', 7.096751428792479)]
DocID: e9f78594-9aac-4f4e-bc5d-d1638616c8ac, Score: 9.390654990355637
Content:
Prompt: ——if you view it the same way. And do you view the U.S. alliance system shifting under your Presidency, away from——
Completion: Who are you with, out of curiosity?

DocID: cf556af9-0e6e-4ead-9cf7-111dfe18dc28, Score: 9.12437

In [10]:
# Check if the dataset contains relevant content
with open('Trump_DB.jsonl', 'r') as f:
    found = False
    for line in f:
        data = json.loads(line)
        prompt = data.get('prompt', '').lower()
        completion = data.get('completion', '').lower()
        if 'economy' in prompt or 'economy' in completion:
            print(f"Prompt: {data.get('prompt', '')}\nCompletion: {data.get('completion', '')}\n")
            found = True
    if not found:
        print("No relevant content found in the dataset.")


Prompt: Well, thank you very much. And we are honored because you selected our show to be the first TV interview you did as president. So it's great that you would us today as well. I know that the Trump team has worked the last four years in the RNC to build a network of people, who you registered and who could get out the vote, especially for today. And so it all comes down to that. But for the person who is undecided on this election day morning, and let's say, they're trying to decide between you and Joe Biden, but they've had a rough year. The pandemic has been tough on everybody. Maybe somebody in their family has lost a job. Maybe they're worried about their healthcare. What do you say to that person? To say, go give me another four years. Let's not switch horses right now.
Completion: Well, it's become pretty standard. We built the greatest Academy in the world. It was horribly interrupted by something that should've never happened. Came in from China, the plague, the plague fr